In [ ]:
import numpy as np
import tensorflow as tf
import sys, os, seaborn, time
import matplotlib.pyplot as plt
sys.path.append('../nnBuilder')
from nnLayer import *
from nnInput import *
from nnTrainer import *
from nnHandler import *
from _nnUtils import *

In [ ]:
in_size=96
out_size=42
data=Layer(type="Constant",file='n43w117.npy',convert_file=True,normalize=True)
data_test=Layer(type="Constant",file='n43w118.npy',convert_file=True,normalize=True)#Bad test set
batch=Layer(x=data,type="Pipeline",batch=128,make_batch=True,
            in_features=[dict(type="Random_Crop",shape=[in_size,in_size]),
                         dict(type="Reshape",shape=[in_size,in_size,1]),
                         dict(type="Whitening"),
                        ])
batch_test=Layer(x=data_test,type="Pipeline",batch=128,make_batch=True,
            in_features=[dict(type="Random_Crop",shape=[in_size,in_size]),
                         dict(type="Reshape",shape=[in_size,in_size,1]),
                        ])
noise=dict(type="Noise",rand_type="normal",scale=0.1,drop_on_test=True)
batch_norm=dict(type="Batch_Norm")

encoder_layers=[]
encoder_layers.append(dict(type="Identity_Label"))
encoder_layers.append(dict(type="Convolution",pad="VALID",window=5,stride=1,size=16,relu=True))
encoder_layers.append(dict(type="Pool",pad="VALID",window=2,stride=2,pool_type="max"))
encoder_layers.append(batch_norm)
encoder_layers.append(noise)
encoder_layers.append(dict(type="Convolution",pad="VALID",window=5,stride=1,size=32,relu=True))
encoder_layers.append(dict(type="Pool",pad="VALID",window=2,stride=2,pool_type="max"))
encoder_layers.append(batch_norm)
encoder_layers.append(noise)
encoder_layers.append(dict(type="Convolution",pad="VALID",window=3,stride=1,size=64,relu=True))
encoder_layers.append(batch_norm)
encoder_layers.append(noise)
encoder_layers.append(dict(type="Convolution",pad="VALID",window=1,stride=1,size=8,relu=False))
encoder_layers.append(batch_norm)
encoder_def=dict(type="Network",layers=encoder_layers)
#encoder=Layer(x=batch,type="Network",layers=encoder_layers)

decoder_layers=[]
decoder_layers.append(dict(type="Convolution",pad="VALID",window=3,stride=1,size=32,relu=True))
encoder_layers.append(batch_norm)
decoder_layers.append(dict(type="Convolution_Transpose",pad="SAME",window=5,stride=2,size=64,relu=True))
encoder_layers.append(batch_norm)
decoder_layers.append(dict(type="Convolution_Transpose",pad="SAME",window=5,stride=2,size=64,relu=True))
encoder_layers.append(batch_norm)
decoder_layers.append(dict(type="Convolution",pad="VALID",window=3,stride=1,size=1,relu=False))
decoder_def=dict(type="Network",layers=decoder_layers)
#decoder=Layer(x=encoder,type="Network",layers=decoder_layers)

crop_def=dict(type="Central_Crop",shape=[out_size,out_size])
crop_def_full=dict(type="Network",layers=[crop_def,dict(type="Label",layer=crop_def)])



network=Layer(x=batch,type="Network",layers=[encoder_def,decoder_def,crop_def])
network_eval=network.copy(x=batch,test=True,share_vars=True)
network_test=network.copy(x=batch_test,test=True,share_vars=True)


labels=Layer(x=network.get_labels(),**crop_def)
labels_eval=Layer(x=network_eval.get_labels(),**crop_def)
labels_test=Layer(x=network_test.get_labels(),**crop_def)

trainer=LabeledTrainer(network=network,optimizer="adam",loss="mean_squared_error",labels=labels)
tester_eval=LabeledTrainer(network=network_eval,loss="mean_squared_error",test=True,labels=labels_eval)
tester=LabeledTrainer(network=network_test,loss="mean_squared_error",test=True,labels=labels_test)


sess=SessManager(data,batch,data_test,batch_test,network,network_eval,network_test,trainer,tester_eval,tester)
sess.start()

In [ ]:
batches_per_step=1000#60000//128 #About 1 epoch
batches_per_eval=50
n_steps=1000
def make_plt():
    %matplotlib notebook
    global fig,ax,train_plot,test_plot,eval_plot
    fig,ax = plt.subplots(1,1)
    train_plot=ax.plot([],[], label="training")[0]
    eval_plot=ax.plot([],[], label="eval")[0]
    test_plot=ax.plot([],[], label="test")[0]
    ax.set_ylim(0,1)
    plt.legend()
    fig.canvas.draw()
    time.sleep(.01)
def update_plt():
    x=[batches_per_step*i for i in range(len(trains))]
    train_plot.set_xdata(x)
    train_plot.set_ydata(trains)
    test_plot.set_xdata(x)
    test_plot.set_ydata(tests)
    eval_plot.set_xdata(x)
    eval_plot.set_ydata(evals)
    ax.set_xlim(0,x[-1])
    ax.set_ylim(0,tests[1]*1.5)
    fig.canvas.draw()
    #time.sleep(.01)
if "trains" not in globals():
    trains=[trainer.eval_loss(n=batches_per_eval)]
    evals=[tester_eval.eval_loss(n=batches_per_eval)]
    tests=[tester.eval_loss()]
make_plt()
for i in range(n_steps):
    '''if i>10:
        r=1e-5
    elif i>5:
        r=1e-4
    else:'''
    r=1e-3
    trainer.train(batches_per_step,keep_rate=0.5,l2reg=1e-6,learn_rate=r)
    trains.append(trainer.eval_loss(n=batches_per_eval)) #Training error
    evals.append(tester_eval.eval_loss(n=batches_per_eval)) 
    tests.append(tester.eval_loss(n=batches_per_eval))#Testing error
    update_plt()

In [ ]:
a=sess.run(batch)

In [ ]:
[a[i].std() for i in range(128)]